# Project 3: Understanding User Behavior
**Project Team: Jude Wentian Zhu, Rohit Barkshi, Rathin Bector**

## Description of Project Files


<span style="color:red">**Need to fill in**</span>

## Project Goal

<span style="color:red">**Need to fill in**</span>

## Summary of Data Pipeline

<span style="color:red">**Need to fill in**</span>

## Data Pipeline Steps

### Step 1: Spin up Docker-Compose and Link Pypark to Jupyter Notebook

1. Spin up the cluster

In [2]:
!docker-compose up -d

Creating network "project_3_default" with the default driver
Creating project_3_zookeeper_1 ... 
Creating project_3_mids_1      ... 
Creating project_3_presto_1    ... 
Creating project_3_cloudera_1  ... 
Creating project_3_kafka_1     ... mdone
Creating project_3_spark_1     ... mdone
ting project_3_spark_1     ... done

2. Exec a bash shell in the spark container on teminal.
```console
docker-compose exec spark bash
```


3. Create a symbolic link from the spark directory to /w205 :
```console
ln -s /w205 w205
```


4. Exit the container
```console
exit
```


5. Check out Hadoop

In [4]:
!docker-compose exec cloudera hadoop fs -ls /tmp/

Found 3 items
drwxrwxrwt   - mapred mapred              0 2016-04-06 02:26 /tmp/hadoop-yarn
drwx-wx-wx   - hive   supergroup          0 2021-08-01 19:22 /tmp/hive
drwxrwxrwt   - mapred hadoop              0 2016-04-06 02:28 /tmp/logs


### Step 2: Launch Kafka and Flask

1. Create a kafka topic called events

In [5]:
!docker-compose exec kafka kafka-topics --create --topic events --partitions 1 --replication-factor 1 --if-not-exists --zookeeper zookeeper:32181

Created topic events.


2. Run flask app on teminal
```console
docker-compose exec mids env FLASK_APP=/w205/project_3/game_api.py flask run --host 0.0.0.0
```


3. Read events from kafka

In [1]:
!docker-compose exec mids kafkacat -C -b kafka:29092 -t events -o beginning -e

% Reached end of topic events [0] at offset 0: exiting


### Step 3: Use Spark to Batch Process Events

In [3]:
# Import Packages
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, from_json
from pyspark.sql.types import StructType, StructField, StringType

In [ ]:
spark = SparkSession \
        .builder \
        .appName("ExtractEventsJob") \
        .getOrCreate()

raw_events = spark \
    .read \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:29092") \
    .option("subscribe", "events") \
    .option("startingOffsets", "earliest") \
    .option("endingOffsets", "latest") \
    .load()

### Step 4: Use Spark to Stream Events